一、导入包

In [3]:
from os import lstat
import scipy
from scipy import io
import cv2 as cv
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import csv

二、数据预处理，生成样本集
2.1、定义函数 mat2csv 将.mat文件转为.csv格式

In [1]:
def mat2csv(mat_pathname,data_features,csv_pathname):
    df=scipy.io.loadmat(mat_pathname)
    features=df[data_features]
    dfdata=pd.DataFrame(data=features)
    dfdata.to_csv(csv_pathname,index=False,header=False)

2.1*、执行函数mat2csv

In [5]:
mat2csv('lb2.mat','lb2','lb2_HEADER.csv')
mat2csv('ah2.mat','ah2','ah2.csv')
#mat2csv('X2.mat','X2','X2.csv')
#mat2csv('Y2.mat','Y2','Y2.csv')


2.2、定义函数csv转list，将csv文件内容逐行读取到列表中

In [6]:
def csv2list(mat_pathname):
    with open(mat_pathname, 'r') as f:
         reader = csv.reader(f)
         list=[]
         for row in reader:
             list+=row
    return list

2.2*、执行csv2list

In [7]:
lb2=csv2list('lb2.csv')
ah2=csv2list('ah2.csv')

In [9]:
print(len(lb2),len(ah2))

159967 159967


2.3、定义函数get_density(),获取并处理图像信息

In [10]:
def get_density(img_path_name):
    #导入
    print(img_path_name)
    img = cv.imread(img_path_name,2)
    img_width=len(img[0])
    img_height=len(img)
    print('原尺寸:',img_width,img_height)
    #标准化尺寸
    img_out=cv.resize(img,(461,347))
    img_width_out=len(img_out[0])
    img_height_out=len(img_out)
    print('标准化尺寸：',img_width_out,img_height_out)
    #扁平化输出
    img_out=img_out.flatten()
    print('扁平化数组长度：',len(img_out),'\n')
    return img_out

2.3*执行函数get_density()

In [11]:
LST=get_density('LST.tif')
forest_height=get_density('Forest_height_2019_SASIAutm49n.tif')
terrain=get_density('terrain.tif')
impervious_surface_percentage=get_density('impervious_surface_percentage_geographic_1000mutm49.tif')
population=get_density('population.tif')




LST.tif
原尺寸: 281 218
标准化尺寸： 461 347
扁平化数组长度： 159967 

Forest_height_2019_SASIAutm49n.tif
原尺寸: 280 217
标准化尺寸： 461 347
扁平化数组长度： 159967 

terrain.tif
原尺寸: 308 278
标准化尺寸： 461 347
扁平化数组长度： 159967 

impervious_surface_percentage_geographic_1000mutm49.tif
原尺寸: 396 361
标准化尺寸： 461 347
扁平化数组长度： 159967 

population.tif
原尺寸: 356 325
标准化尺寸： 461 347
扁平化数组长度： 159967 



2.4、定义函数build_sample()，用来生成样本集，存入sample.csv

In [12]:
def build_sample():
    dict={'ah2':ah2,'lb2':lb2,'population':population,\
        'terrain':terrain,'forest_height':forest_height,\
        'impervious_surface_percentage':impervious_surface_percentage,\
        'LST':LST}
        
    df=pd.DataFrame(dict)
    df.to_csv('sample.csv',index=False)

2.4*、执行 build_sample()

In [13]:
build_sample()

三、导入样本集

In [14]:
path = 'sample.csv'
data = pd.read_csv(path)

In [15]:
#观察样本状况
print(data.head())
print(data.describe())
#data.plot(kind='scatter',x='ah2',y='LST',figsize=(12,8))
#plt.show()
#set X (training data) and y (target variable)


   ah2  lb2  population  terrain  forest_height  \
0  0.0  0.0   317.87192      658             17   
1  0.0  0.0   317.87195      714             16   
2  0.0  0.0   317.87195      717             16   
3  0.0  0.0   317.87195      509             13   
4  0.0  0.0   317.87192      361             11   

   impervious_surface_percentage  LST  
0                            255  0.0  
1                            255  0.0  
2                            255  0.0  
3                            255  0.0  
4                            255  0.0  
                ah2           lb2    population        terrain  forest_height  \
count  1.599670e+05  1.599670e+05  1.599670e+05  159967.000000  159967.000000   
mean   3.563926e+05  6.013415e+04 -8.635121e+37   13269.607357      26.146812   
std    1.008648e+06  1.841795e+05  1.466689e+38   23219.858147      36.146018   
min    0.000000e+00  0.000000e+00 -3.402823e+38       1.000000       0.000000   
25%    0.000000e+00  0.000000e+00 -2.399272e+38 

3.1从样本集中将标签lable划分出来

In [56]:
cols = data.shape[1]
X = data.iloc[:,0:cols-1]
y = data.iloc[:,cols-1:cols]


In [57]:
#查看划分状况
print(X.head())
print(y.head())
print(X.iloc[0:4, [0, 1, 2, -1]])

   ah2  lb2  population  terrain  forest_height  impervious_surface_percentage
0  0.0  0.0   317.87192      658             17                            255
1  0.0  0.0   317.87195      714             16                            255
2  0.0  0.0   317.87195      717             16                            255
3  0.0  0.0   317.87195      509             13                            255
4  0.0  0.0   317.87192      361             11                            255
   LST
0  0.0
1  0.0
2  0.0
3  0.0
4  0.0
   ah2  lb2  population  impervious_surface_percentage
0  0.0  0.0   317.87192                            255
1  0.0  0.0   317.87195                            255
2  0.0  0.0   317.87195                            255
3  0.0  0.0   317.87195                            255


3.2随机划分训练集和测试集

In [58]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X.values,y.values,test_size=0.2)

In [59]:
type(X_test)

numpy.ndarray

四、线性回归

In [60]:
#linear regression
from sklearn import linear_model
start=time.time()

#fit 回归模型
model=linear_model.LinearRegression()
model.fit(X_train,y_train)
print('intercept_:',model.intercept_)
print('coef_:','\n',model.coef_)

from sklearn.metrics import mean_squared_error,r2_score
print('Mean squared error:',mean_squared_error(y_test,model.predict(X_test)))
print ('Variance score：',r2_score(y_test,model.predict(X_test)))
print('score：',model.score(X_test,y_test))
print('time usage:',time.time()-start,'s')

intercept_: [277.13772686]
coef_: 
 [[ 2.17484956e-68  0.00000000e+00  5.51081229e-37 -7.32765971e-71
  -8.32946657e-74  5.35865857e-74]]
Mean squared error: 6875.154201142223
Variance score： 0.4974860979426504
score： 0.4974860979426504
time usage: 0.019991159439086914 s


五、梯度提升拟合

In [ ]:
#light GBM regression
import lightgbm as lgb
print('################### light GBM regression ##########')
start=time.time()
# #######################################################################
#fit 回归模型
#model = lgb.LGBMRegressor(num_leaves=31,learning_rate=0.05,n_estimators=20)
model = lgb.LGBMRegressor()
model.fit (X_train,y_train.ravel())

#look at the result
from sklearn.metrics import mean_squared_error,r2_score
print('Mean squared error；',mean_squared_error(y_test,model.predict(X_test)))
print ('Variance score：',r2_score(y_test,model.predict(X_test)))
print('score：',model.score(X_test,y_test))
print('time usage:',time.time()-start,'s')

################### light GBM regression ##########
Mean squared error； 1801.5805695714844
Variance score： 0.8683894690773961
score： 0.8683894690773961
time usage: 0.31507134437561035 s


神经网络

In [66]:
import torch 
from torch import nn
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from d2l import torch as d2l
import random
from torch.utils import data

In [62]:
train_features = torch.tensor(X_train, dtype=d2l.float32)
test_features = torch.tensor(X_test, dtype=d2l.float32)
train_labels = torch.tensor(y_train, dtype=d2l.float32)
test_labels = torch.tensor(y_test, dtype=d2l.float32)

In [63]:
type(train_features)

torch.Tensor

迭代器

In [86]:
def load_array(data_arrays, batch_size, is_train=True):  
    """构造一个PyTorch数据迭代器。"""
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)

batch_size = 300
train_iter = load_array((train_features, train_labels), batch_size)
test_iter = load_array((test_features, test_labels), batch_size)

In [87]:
next(iter(train_iter))

[tensor([[ 0.0000e+00,  0.0000e+00, -2.6029e+38,  5.5537e+04,  1.0100e+02,
           3.0000e+00],
         [ 0.0000e+00,  0.0000e+00, -3.4028e+38,  5.5537e+04,  1.0100e+02,
           2.2900e+02],
         [ 0.0000e+00,  0.0000e+00, -3.1949e+38,  7.8160e+03,  8.0000e+00,
           0.0000e+00],
         ...,
         [ 0.0000e+00,  0.0000e+00,  2.2915e+02,  6.8000e+02,  9.0000e+00,
           0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  1.1412e+02,  3.9000e+01,  6.0000e+00,
           0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  1.2638e+02,  1.2000e+01,  2.8000e+01,
           0.0000e+00]]),
 tensor([[  0.0000],
         [  0.0000],
         [291.1706],
         [ 14.4488],
         [  0.0000],
         [  0.0000],
         [293.0214],
         [290.4715],
         [291.6615],
         [293.6688],
         [290.0199],
         [289.9095],
         [291.0002],
         [290.8986],
         [293.6282],
         [  0.0000],
         [  0.0000],
         [289.8026],
         [  

模型

In [98]:
net = nn.Sequential(nn.Flatten(),nn.Linear(6, 3), nn.ReLU(),
                    nn.Linear(3, 1) )
net[1].weight.data.normal_(0, 0.01)
net[1].bias.data.fill_(0)

tensor([0., 0., 0.])

定义损失和优化器

In [107]:
batch_size, lr, num_epochs = 159968, 0.01, 10
loss = nn.MSELoss()
trainer = torch.optim.SGD(net.parameters(), lr=lr)


In [108]:
num_epochs = 3
for epoch in range(num_epochs):
    for X, y in train_iter:
        l = loss(net(X) ,y)
        trainer.zero_grad()
        l.backward()
        trainer.step()
    l = loss(net(train_features), train_labels)
    print(f'epoch {epoch + 1}, loss {l:f}')

epoch 1, loss nan
epoch 2, loss nan
epoch 3, loss nan


In [93]:
d2l.train_ch3(net, train_iter, test_iter, loss, num_epochs, trainer)

AssertionError: nan

In [73]:

for epoch in range(num_epochs):
    for X, y in train_iter:
        l = loss(net(X) ,y)
        trainer.zero_grad()
        l.backward()
        trainer.step()
    l = loss(net(features), labels)
    print(f'epoch {epoch + 1}, loss {l:f}')

NameError: name 'features' is not defined